In [1]:
import sys, time, pickle
import torch

sys.path.insert(0, '../../Utils')
from global_constants import gpu_device

sys.path.insert(0, '../')
from BERT_models import BERT_SST2_MODEL

from _utils import sample_random_glue_sst2, get_continuation_mapping, \
                    get_continuous_attributions, get_continuous_raw_inputs, \
                    collect_info_for_metric, save_info, download_HTML

In [2]:
sst2_data_raw, targets, idxs = sample_random_glue_sst2()

Reusing dataset glue (/home/user/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /home/user/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-3b24abff24d1d8c0.arrow
Loading cached processed dataset at /home/user/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-5960909ab3834668.arrow


In [3]:
model = BERT_SST2_MODEL()
tokenizer = model.tokenizer

In [4]:
model_out_list, raw_attr_list, conti_attr_list, raw_input_list = [], [], [], []

In [5]:
from captum.attr import IntegratedGradients
from captum.attr import visualization 

In [6]:
# lime = Lime(forward_func=model.forward)
IG = IntegratedGradients(model)

In [7]:
def generate_record(raw_review, target):
    #tokenizer operations
    tokenized = tokenizer(raw_review, truncation=True, return_offsets_mapping=True)
    offset_mapping = tokenized['offset_mapping']
    conti_map = get_continuation_mapping(offset_mapping)
    input_ids = torch.tensor(tokenized['input_ids']).unsqueeze(0).to(gpu_device)
    detokenized = [t.replace('#', '') for t in tokenizer.convert_ids_to_tokens(input_ids[0])]

    #feeding input forward 
    input_emb = model.get_embeddings(input_ids)
    pred_prob = model(input_emb).item()
    
    #categorizing results
    pred_class = 'Pos' if pred_prob > 0.5 else 'Neg' 
    true_class = 'Pos' if target > 0.5 else 'Neg' 
    
    #attribution algorithm working
    attribution, delta = IG.attribute(input_emb, n_steps=50, return_convergence_delta=True)
    word_attributions = attribution.sum(dim=2).squeeze(0)
    word_attributions /= torch.norm(word_attributions)
    attr_score = torch.sum(word_attributions)
    attr_class = 'Pos' if attr_score > 0.5 else 'Neg'
    convergence_score = delta
    
    
    #re-organizing tensors and arrays because words get split down
    conti_attr = get_continuous_attributions(conti_map, word_attributions)
    raw_input = get_continuous_raw_inputs(conti_map, detokenized)

#     print(f'word attributions {word_attributions}')
#     print(f'pred_prob {pred_prob}')
#     print(f'pred_class {pred_class}')
#     print(f'true_class {true_class}')
#     print(f'attribution {attribution}')
#     print(f'attr_class {attr_class}')
#     print(f'attr_score {attr_score}')
#     print(f'raw_input {raw_input}')

        
#     collect info for metrics later
    collect_info_for_metric(model_out_list, pred_prob, raw_attr_list, attribution, conti_attr_list, conti_attr, raw_input_list, raw_input)
        
    
    visual_record = visualization.VisualizationDataRecord(word_attributions=conti_attr,
                                                         pred_prob=pred_prob,
                                                         pred_class=pred_class,
                                                         true_class=true_class,
                                                         attr_class=attr_class,
                                                         attr_score=attr_score,
                                                         raw_input_ids=raw_input,
                                                         convergence_score=convergence_score)
        
        
    return visual_record
      
    

In [8]:
for i, (datum_raw, target) in enumerate(zip(sst2_data_raw, targets), start=1):
    print(f'Raw review: {datum_raw}')
    print(f'GT target: {target}')
    visual_record=generate_record(datum_raw, target)
    print(visualization.visualize_text([visual_record]))


Raw review: its oscar nomination 
GT target: 1
word attr tensor([ 0.2417,  0.0591, -0.0531, -0.0784,  0.9639], device='cuda:0',
       dtype=torch.float64, grad_fn=<AsStridedBackward0>)
conti attr [tensor(0.2417, device='cuda:0', dtype=torch.float64, grad_fn=<UnbindBackward0>), tensor(0.0591, device='cuda:0', dtype=torch.float64, grad_fn=<UnbindBackward0>), tensor(-0.0531, device='cuda:0', dtype=torch.float64,
       grad_fn=<UnbindBackward0>), tensor(-0.0784, device='cuda:0', dtype=torch.float64,
       grad_fn=<UnbindBackward0>), tensor(0.9639, device='cuda:0', dtype=torch.float64, grad_fn=<UnbindBackward0>)]
detokenized ['[CLS]', 'its', 'oscar', 'nomination', '[SEP]']
len conti_raw 5
conti_raw ['[CLS]', 'its', 'oscar', 'nomination', '[SEP]']


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
Pos,Pos (0.96),Pos,1.13,[CLS] its oscar nomination [SEP]


<IPython.core.display.HTML object>
Raw review: shenanigans and slapstick 
GT target: 1
word attr tensor([ 0.1096,  0.0312, -0.0942, -0.0518, -0.1100,  0.4153, -0.4911, -0.6989,
         0.2475], device='cuda:0', dtype=torch.float64,
       grad_fn=<AsStridedBackward0>)
conti attr [tensor(0.1096, device='cuda:0', dtype=torch.float64, grad_fn=<UnbindBackward0>), tensor(-0.0758, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>), tensor(0.4153, device='cuda:0', dtype=torch.float64, grad_fn=<UnbindBackward0>), tensor(-0.5950, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>), tensor(0.2475, device='cuda:0', dtype=torch.float64, grad_fn=<UnbindBackward0>)]
detokenized ['[CLS]', 'shen', 'ani', 'gan', 's', 'and', 'slap', 'stick', '[SEP]']
len conti_raw 5
conti_raw ['[CLS]', 'shenanigans', 'and', 'slapstick', '[SEP]']


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
Pos,Neg (0.03),Neg,-0.64,[CLS] shenanigans and slapstick [SEP]


<IPython.core.display.HTML object>
Raw review: an unsettling sight , 
GT target: 0
word attr tensor([ 0.3754,  0.6306, -0.4800, -0.2396,  0.2948,  0.0930,  0.2775, -0.0315],
       device='cuda:0', dtype=torch.float64, grad_fn=<AsStridedBackward0>)
conti attr [tensor(0.3754, device='cuda:0', dtype=torch.float64, grad_fn=<UnbindBackward0>), tensor(0.6306, device='cuda:0', dtype=torch.float64, grad_fn=<UnbindBackward0>), tensor(-0.0325, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>), tensor(0.0930, device='cuda:0', dtype=torch.float64, grad_fn=<UnbindBackward0>), tensor(0.2775, device='cuda:0', dtype=torch.float64, grad_fn=<UnbindBackward0>), tensor(-0.0315, device='cuda:0', dtype=torch.float64,
       grad_fn=<UnbindBackward0>)]
detokenized ['[CLS]', 'an', 'un', 'sett', 'ling', 'sight', ',', '[SEP]']
len conti_raw 6
conti_raw ['[CLS]', 'an', 'unsettling', 'sight', ',', '[SEP]']


...

<IPython.core.display.HTML object>
Raw review: unrewarding 
GT target: 0
word attr tensor([ 0.4536, -0.2069, -0.3012, -0.1737,  0.7941], device='cuda:0',
       dtype=torch.float64, grad_fn=<AsStridedBackward0>)
conti attr [tensor(0.4536, device='cuda:0', dtype=torch.float64, grad_fn=<UnbindBackward0>), tensor(-0.2139, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>), tensor(0.7941, device='cuda:0', dtype=torch.float64, grad_fn=<UnbindBackward0>)]
detokenized ['[CLS]', 'un', 'rew', 'arding', '[SEP]']
len conti_raw 3
conti_raw ['[CLS]', 'unrewarding', '[SEP]']


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
Neg,Neg (0.14),Pos,0.57,[CLS] unrewarding [SEP]


<IPython.core.display.HTML object>
Raw review: old-hat 
GT target: 0
word attr tensor([-0.8564, -0.1489, -0.0858,  0.0700, -0.4818], device='cuda:0',
       dtype=torch.float64, grad_fn=<AsStridedBackward0>)
conti attr [tensor(-0.8564, device='cuda:0', dtype=torch.float64,
       grad_fn=<UnbindBackward0>), tensor(-0.0237, device='cuda:0', dtype=torch.float64, grad_fn=<DivBackward0>), tensor(-0.4818, device='cuda:0', dtype=torch.float64,
       grad_fn=<UnbindBackward0>)]
detokenized ['[CLS]', 'old', '-', 'hat', '[SEP]']
len conti_raw 3
conti_raw ['[CLS]', 'old-hat', '[SEP]']


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
Neg,Neg (0.05),Neg,-1.50,[CLS] old-hat [SEP]


<IPython.core.display.HTML object>


In [9]:
file_name_base = 'Integrated_Gradients'

In [10]:
save_info(idxs, sst2_data_raw, targets, model_out_list, raw_attr_list, conti_attr_list, raw_input_list, 
          fname=f'../MethodOutputs/{file_name_base}_out.pkl')

{'indices': [27798,
  28244,
  48083,
  8795,
  91,
  20682,
  59890,
  41763,
  19795,
  9957,
  28334,
  55664,
  16438,
  18031,
  9297,
  66913,
  9289,
  6789,
  25294,
  33294,
  29699,
  41960,
  6554,
  8772,
  17921,
  8027,
  49091,
  54186,
  15080,
  22382,
  23856,
  9464,
  23831,
  59434,
  17862,
  24584,
  26216,
  63094,
  55787,
  3993,
  25463,
  17540,
  51128,
  46224,
  32656,
  30105,
  28646,
  17011,
  7812,
  48236],
 'raw_data': ['its oscar nomination ',
  'shenanigans and slapstick ',
  'an unsettling sight , ',
  'the climactic hourlong cricket match ',
  'alternating between facetious comic parody and pulp melodrama , this smart-aleck movie ... tosses around some intriguing questions about the difference between human and android life ',
  'to be a part of that elusive adult world ',
  'emotional power ',
  'reminds you of why animation is such a perfect medium for children , because of the way it allows the mind to enter and accept another world ',
  'un

In [11]:
download_HTML(cur_file_name=f'{file_name_base}_SST2_BERT.ipynb',
              out_file_name=f'{file_name_base}_SST2_BERT.html')